In [3]:
from datetime import date
from blog.models import Post
from organizer.models import Tag, Startup, NewsLink

# Basic interaction with Django models

In [ ]:
edut = Tag(name="Education", slug="education")

In [ ]:
edut

In [ ]:
edut.name

In [ ]:
edut.save() #saves the data model into the database

In [ ]:
edut.delete() #deletes data model from database

In [ ]:
edut #still in memory

# Creation and Destruction with Model Managers

In [ ]:
type(Tag.objects) #model manager

In [ ]:
#Using the model manager you are creating the object directly in the database
#Creates one object in one query
Tag.objects.create(name='Video Games', slug='video-games')

In [ ]:
#You can also create various objects at one time
#Creates multiple objects in one query
Tag.objects.bulk_create([
    Tag(name="Django", slug='django'),
    Tag(name="Mobile", slug='mobile'),
    Tag(name="Web", slug='web')
])

In [ ]:
#Quering DB for all objects. Try not to do this directly in the app, as it can be quiet expensive.
#Notice the response is a QuerySet an that the ordering is alphabetical, just like we specify in the Meta class
Tag.objects.all()

In [ ]:
#Even is we get a QuerySet, we can use it as an array to get the index object 
Tag.objects.all()[0]

In [ ]:
#This type of Django Queryset. Ability to create a query set without touching database
#and modiying the query set before launching the query on the DB.
type(Tag.objects.all())

In [ ]:
#managers are not accesible from the model instance, only to model classes
try:
    edut.objects
except AttributeError as e:
    print(e)

# Methods of Data Retrieval

In [ ]:
Tag.objects.all()

In [ ]:
#Model Manager can count the objects in DB
Tag.objects.count()

## the get method

In [ ]:
#Get method matching slug, This type of methods can actually creates SQL
#Is one of the methods that DOESN'T return a query set. 
#It returns an MODEL instance 
Tag.objects.get(slug='django')

In [ ]:
type(Tag.objects.all())

In [ ]:
type(Tag.objects.get(slug='django'))

In [ ]:
#case sensitive
try:
    Tag.objects.get(slug='Django')
except Tag.DoesNotExist as e:
    print(e)
    

In [ ]:
#we can modify the SQL query
#slug__iexact: where i stands for case insensitive and exact for the exact query text.
Tag.objects.get(slug__iexact='DJANGO')

In [ ]:
#slug__istartswith: case insensitive, starting with 'DJ'
Tag.objects.get(slug__istartswith='DJ')

In [ ]:
#slug__contains: object with slug containing 'an'
Tag.objects.get(slug__contains='an')

In [ ]:
#get always return a single object
try:
    Tag.objects.get(slug__contains='o')
except Tag.MultipleObjectsReturned as e:
    print(e)

## The filter method

In [ ]:
#unlike get, it can fetch multiple objects
Tag.objects.filter(slug__contains='o')

In [ ]:
#filter returns a query set
type(Tag.objects.filter(slug__contains='o'))

## Chaining Calls

In [ ]:
#Fetch the objects and then order in descending
Tag.objects.filter(slug__contains='o').order_by('-name')

In [ ]:
#first we call order_by on the manager
Tag.objects.order_by('-name')

In [ ]:
#Now we call the filter on the manager, and order the resulting queryset
Tag.objects.filter(slug__contains='e').order_by('-name')

## values and values_list

In [ ]:
Tag.objects.values_list()

In [ ]:
type(Tag.objects.values_list())

In [ ]:
#you can specify the return of the value list by:
Tag.objects.values_list('name', 'slug')

In [ ]:
#notice this will return a tuple
Tag.objects.values_list('name')

In [ ]:
#if we need to remove the tuple from the way we can use flat
Tag.objects.values_list('name', flat=True)

In [ ]:
type(Tag.objects.values_list('name', flat=True))

# Data in Memory vs Data in Database

In [19]:
jb = Startup.objects.create(
    name='JamBom Software',
    slug='jamBom-software',
    contact='django@jbsofware.com',
    description='Web and Mobile Consulting. \n',
    founded_date = date(2013, 1, 18),
    website='https://jambomsw.com/'
)
jb

IntegrityError: UNIQUE constraint failed: organizer_startup.slug

In [21]:
jb = Startup.objects.get(slug__contains='jamBom-software')
jb.founded_date

datetime.date(2013, 1, 18)

In [ ]:
jb.founded_date = date(2014,1,1)
#we are not calling save
jb.founded_date

In [ ]:
#get the object from the database
jb = Startup.objects.get(slug='jamBom-software')
jb.founded_date

# Connecting Data through Relations

In [4]:
#create a new post
djt = Post.objects.create( 
    title ='Django Training',
    slug ='django-training',
    text = ("Learn Django in a classroom setting"
           "with jambom software"),
    pub_date= date(2020,1,1)
)

djt

<Post: Django Training on 2020-01-01>

In [7]:
#modifed pub date and save it to the DB
djt.pub_date = date(2020,2,2)
djt.save()
djt.refresh_from_db()
djt

<Post: Django Training on 2020-02-02>

In [8]:
#we have 2 reletions in post: tags and startups
type(djt.tags)

django.db.models.fields.related_descriptors.create_forward_many_to_many_manager.<locals>.ManyRelatedManager

In [9]:
type(djt.startups)

django.db.models.fields.related_descriptors.create_forward_many_to_many_manager.<locals>.ManyRelatedManager

In [10]:
#you can interact with this relations as you would using a manager
djt.startups.all()

<QuerySet []>

In [11]:
djt.tags.all()

<QuerySet []>

In [12]:
#fetch a tag object
django = Tag.objects.get(slug__contains='django')
#connect the tag to the post 
djt.tags.add(django)
#get the related tags in this post
djt.tags.all()

<QuerySet [<Tag: Django>]>

In [18]:
#Given a Tag, use related_name=blog_post to fetch all blog post related to this tag
django.blog_posts.all()  #reverse relationship

<QuerySet [<Post: Django Training on 2020-02-02>, <Post: Django Training on 2020-02-02>, <Post: Django Training on 2020-02-02>]>

In [28]:
#Connect startup(jb) to a Tag(django) using the "startup_set" reserved keyword.
django.startup_set.add(jb)
#(Tag)<- Startups 
django.startup_set.all()  #reverse relationship

<QuerySet [<Startup: JamBom Software>]>

In [46]:
#Given a startup (jb) check all related tags
jb.tags.all()

<QuerySet [<Tag: Django>]>

In [43]:
#Add a startup(jb) to a Given a Post(djt) 
djt.startups.add(jb)
#Check startups related to a post (djt)
djt.startups.all()

<QuerySet [<Startup: JamBom Software>]>

In [44]:
#using the related_name=blog_posts, search Posts related to a tag
django.blog_posts.all()

<QuerySet [<Post: Django Training on 2020-02-02>, <Post: Django Training on 2020-02-02>, <Post: Django Training on 2020-02-02>]>

In [45]:
django.startup_set.all()

<QuerySet [<Startup: JamBom Software>]>

In [47]:
jb.blog_posts.all()

<QuerySet [<Post: Django Training on 2020-02-02>]>

In [48]:
jb.tags.all() #the forward relation

<QuerySet [<Tag: Django>]>